## Context
The previous dataset didn't provide the sufficient details for the feature.
This web scraper will create a dataset with the results for each party in each parish ('freguesia') in Portugal (inland and autonomous territories) as well as the votes from the Europe and Outside of Europe circles

In [1]:
%pip intall pandas
import pandas as pd

ERROR: unknown command "intall" - maybe you meant "install"



Note: you may need to restart the kernel to use updated packages.


In [2]:
# Base url
site = 'https://www.legislativas2024.mai.gov.pt/assets/static'
json = '.json'

# Local path
tchild = site +'/territory-children/territory-children-' # children territory codes url
tresult = site +'/territory-results/territory-results-' # voting results url

portugal = {
                'compensation': [False], 
                'name': ['Portugal'], 
                'territoryKey': ['LOCAL-500000'],
                'distrito': False,
                'concelho': False,
                'level': ['pais']
            }
codes = pd.DataFrame(portugal) # creates codes dataframe, and populates with highest-tier level

freguesias_results = pd.DataFrame() # creates voting results dataframe

In [3]:
# Recursively drills down to freguesias, then gets results
def get_results(locals, level):
    for i, row in locals.iterrows():
        get_codes(row, level)
#         match level:
#             case 'freguesia':
#                 freguesias_results = freguesias_results.append(get_json(row['territoryKey']))
#             case 'concelho':
#                 get_results(get_json(row['territoryKey']), 'freguesia')
#             case 'distrito':
#                 get_results(get_json(row['territoryKey']), 'conselho')
            

# Saves json from url
def get_json(territoryKey):
    url = tresult + territoryKey + json
    return pd.read_json(url)

# Populates codes dataframe
def get_codes(row, district, county, level):
    return  {
                'compensation':     row['compensation'],
                'name':             row['name'], 
                'territoryKey':     row['territoryKey'],
                'distrito':         district,
                'concelho':         county,
                'level':            level
            }

In [4]:
# Drill down to districts
districts = pd.read_json(tchild + codes['territoryKey'][0] + json)

district_codes = pd.DataFrame(get_codes(districts, False, False, 'distrito'))

In [5]:
county_codes = pd.DataFrame()

# Drill down to counties
for row in district_codes.iterrows():
    tkey = row[1]['territoryKey']
    dist = row[1]['name']
    county = False
    counties = pd.read_json(tchild + tkey + json)
    codes = pd.DataFrame(get_codes(counties, dist, county, 'concelho'))
    county_codes = pd.concat([county_codes, codes])

county_codes.reset_index(drop=True)

,compensation,name,territoryKey,distrito,concelho,level
0,False,Angra do Heroísmo,LOCAL-430100,Açores,False,concelho
1,False,Calheta,LOCAL-450100,Açores,False,concelho
2,False,Corvo,LOCAL-490100,Açores,False,concelho
3,False,Horta,LOCAL-470100,Açores,False,concelho
4,False,Lagoa,LOCAL-420100,Açores,False,concelho
...,...,...,...,...,...,...
303,False,Tarouca,LOCAL-182000,Viseu,False,concelho
304,False,Tondela,LOCAL-182100,Viseu,False,concelho
305,False,Vila Nova de Paiva,LOCAL-182200,Viseu,False,concelho
306,False,Viseu,LOCAL-182300,Viseu,False,concelho


In [6]:
parish_codes = pd.DataFrame()

# Drill down to counties
for row in county_codes.iterrows():
    tkey = row[1]['territoryKey']
    dist = row[1]['distrito']
    county = row[1]['name']
    parishes = pd.read_json(tchild + tkey + json)
    codes = pd.DataFrame(get_codes(parishes, dist, county, 'freguesia'))
    parish_codes = pd.concat([parish_codes, codes])
    
parish_codes = parish_codes.reset_index(drop=True)

In [7]:
parish_codes

,compensation,name,territoryKey,distrito,concelho,level
0,False,Altares,LOCAL-430101,Açores,Angra do Heroísmo,freguesia
1,False,Angra (Nossa Senhora da Conceição),LOCAL-430102,Açores,Angra do Heroísmo,freguesia
2,False,Angra (Santa Luzia),LOCAL-430103,Açores,Angra do Heroísmo,freguesia
3,False,Angra (São Pedro),LOCAL-430104,Açores,Angra do Heroísmo,freguesia
4,False,Angra (Sé),LOCAL-430105,Açores,Angra do Heroísmo,freguesia
...,...,...,...,...,...,...
3087,False,Fornelo do Monte,LOCAL-182407,Viseu,Vouzela,freguesia
3088,False,Queirã,LOCAL-182409,Viseu,Vouzela,freguesia
3089,False,São Miguel do Mato,LOCAL-182410,Viseu,Vouzela,freguesia
3090,False,Ventosa,LOCAL-182411,Viseu,Vouzela,freguesia
